# 1. Traveling salesman (Problema comis-voiajorului)

Un poștaș pleacă în fiecare dimineață de la oficiul poștal cu scrisorile pe care trebuie să livreze în ziua respectivă și adresele acestora.

Folosește algoritmul IDA* pentru a identifica cel mai scurt drum pe care îl poate face astfel încât să treacă fix o dată prin fiecare locație și să se întoarcă la oficiul poștal la finalul zilei.

In [2]:
class Nod_Oras:
    def __init__(self, informatie, parinte=None, g=0, h=0):
        """
        :param informatie: informatia (numele orasului) nodului curent
        :param parinte: pointer catre nodul parinte
        :param g: costul drumului de la nodul start pana la nodul curent
        :param h: estimarea costului de la nodul curent pana la final
        """
        self.informatie = informatie
        self.parinte = parinte
        self.g = g
        self.h = h
        self.f = g + h

    def __repr__(self):
        return str(self.informatie)

    def __str__(self):
        return repr(self) + ' (' + ' -> '.join([repr(x) for x in self.drumRadacina()]) + ')'

    def __eq__(self, other):
        return self.informatie == other.informatie

    def __lt__(self, other):
        return self.f < other.f or (self.f == other.f and self.g > other.g)

    def drumRadacina(self):
        """Calculeaza lista nodurilor de la radacina pana la nodul curent."""
        if self.parinte is None:
            return [self]
        return self.parinte.drumRadacina() + [self]

    def vizitat(self):
        """Verifica daca nodul apare de mai multe ori in drumul de la el pana la radacina."""
        return len([n for n in self.drumRadacina() if n == self]) > 1

In [8]:
class Graf_TS:
    def __init__(self, nodStart, muchii):
        """
        :param nodStart: numele orasului de start
        :param muchii: lista de muchii sub forma (oras_start, oras_destinatie, cost)
        """
        self.nodStart = Nod_Oras(nodStart)
        self.muchii = muchii

        # Calculam multimea de orase din graf
        self.orase = set()
        for n1, n2, cost in muchii:
            self.orase.add(n1)
            self.orase.add(n2)
        self.orase.add(nodStart)

        # Pre-calculam costul minim al unei muchii (folosit in estimare)
        self.minEdge = min(cost for (_, _, cost) in muchii)


    def estimeaza_h(self, informatie):
        """
        Functia de estimare (heuristica) pentru un nod dat.
        :param informatie: numele orasului pentru care estimam h
        :return: o estimare a costului ramas pentru a finaliza turul
        """
        oras_curent = informatie
        drum = [informatie]

        nr_nevizitate = len(self.orase - set(drum))
        if nr_nevizitate == 0:
            # Daca toate orasele au fost vizitate, estimam costul de intoarcere la start
            for n1, n2, cost in self.muchii:
                if n1 == oras_curent and n2 == self.nodStart.informatie:
                    return cost
            return 0
        # Estimare: numarul de orase nevizitate + 1 (pentru intoarcere) inmultit cu costul minim
        return (nr_nevizitate + 1) * self.minEdge


    def scop(self, nod):
        """
        Nodul este scop daca drumul contine toate orasele si se incheie in orasul de start.
        :param nod: nodul curent
        :return: True daca nodul reprezinta o solutie, False altfel
        """
        drum = [n.informatie for n in nod.drumRadacina()]
        return (len(drum) == len(self.orase) + 1 and drum[-1] == self.nodStart.informatie)


    def succesori(self, nod):
        """
        Returneaza succesorii nodului, evitand ciclurile (nu se re-viziteaza un oras, cu exceptia
        intoarcerii la nodul de start, care este permisa doar daca s-au vizitat toate orasele).
        :param nod: nodul curent
        :return: lista succesorilor nodului
        """
        succesori = []
        drum = [n.informatie for n in nod.drumRadacina()]
        for nod1, nod2, cost in self.muchii:
            if nod1 != nod.informatie:
                continue

            # Permitem întoarcerea la nodul de start doar dacă toate oraşele au fost vizitate
            if len(drum) == len(self.orase):
              if nod2 == self.nodStart.informatie:
                    nou_nod = Nod_Oras(nod2, nod, nod.g + cost, self.estimeaza_h(nod2))
                    succesori.append(nou_nod)
            else:
                if nod2 in drum:
                    continue
                nou_nod = Nod_Oras(nod2, nod, nod.g + cost, self.estimeaza_h(nod2))
                succesori.append(nou_nod)
        return succesori


    def search(self, nod, bound):
        """
        Functia recursiva folosita de IDA*.
        :param nod: nodul curent
        :param bound: limita curenta a lui f
        :param graf: graful problemei
        :return: fie un nod scop, fie noua limita minima pentru f
        """
        f = nod.g + nod.h
        if f > bound:
            return f
        if self.scop(nod):
            return nod
        min_bound = float('inf')
        for succesor in self.succesori(nod):
            v = self.search(succesor, bound)
            if isinstance(v, Nod_Oras):
                return v
            if v < min_bound:
                min_bound = v
        return min_bound


    def ida_star(self):
        """
        Algoritmul IDA* care iterativ creste limita 'bound' pana gaseste solutia.
        :param graf: obiectul de tip Graf
        :return: nodul scop daca este gasit, altfel None
        """
        start = graf.nodStart
        graf.nodStart.h = graf.estimeaza_h(start.informatie)
        start.f = start.g + start.h
        bound = start.f

        while True:
            #print("Limita actuala:", bound)
            v = self.search(start, bound)
            if isinstance(v, Nod_Oras):
                return v  # solutia a fost gasita
            if v == float('inf'):
                return None  # nu exista solutie
            bound = v  # actualizeaza limita pentru iteratia urmatoare


# Definim lista de muchii (exemplu clasic cu 4 orase: A, B, C, D)
muchii = [
    ('A', 'B', 10),
    ('A', 'C', 15),
    ('A', 'D', 20),
    ('B', 'A', 10),
    ('B', 'C', 35),
    ('B', 'D', 25),
    ('C', 'A', 15),
    ('C', 'B', 35),
    ('C', 'D', 30),
    ('D', 'A', 20),
    ('D', 'B', 25),
    ('D', 'C', 30)
]

graf = Graf_TS('A', muchii)
solutie = graf.ida_star()
if solutie:
    drum = solutie.drumRadacina()
    print("\nSolutie:")
    print(" -> ".join(str(n.informatie) for n in drum), "\nCost total =", solutie.g)
else:
    print("Nu s-a gasit nicio solutie.")


Solutie:
A -> B -> D -> C -> A 
Cost total = 80


# 2. Connect four

Implementează jocul și joacă împotriva calculatorului. Folosește Alpha Beta Pruning pentru a stabili mutările acestuia.

In [ ]:
# pentru manipulare eleganta de matrici
import numpy as np

class Game:
  MAX = 'R'
  MIN = 'Y'
  GOL = '.'

  @classmethod
  def jucator_opus(self, jucator):
    return Game.MIN if jucator == Game.MAX else Game.MAX

In [ ]:
import numpy as np


class Nod:
  def __init__(self, informatie=np.array([[Game.GOL] * 7 for _ in range(6)]), parinte=None, jucator=Game.MAX):
    ''' Constructorul clasei Nod '''
    self.informatie = informatie
    self.parinte = parinte
    self.jucator = jucator
    self.estimare_scor = 0
    #self.lista_succesori = []

  def __repr__(self):
    return str(self.informatie)

  def __str__(self):
    return '\n-------------\n'.join(['|'.join(self.informatie[i]) for i in range(6)])

  def __eq__(self, cls):
     return self.informatie == cls.informatie

  def __lt__(self, cls):
    if cls is None:
      return self
    return self.estimare_scor <= cls.estimare_scor

  def succesori(self):
    ''' Calculeaza lista succesorilor directi ai starii curente.
    :return: lista starilor admisibile
    '''
    succesori = []
    for col in range(7):
      for row in range(5, -1, -1): # parcurgem coloanele de jos in sus
        if self.informatie[row, col] != Game.GOL:
          continue
        # punem un disc in varful coloanei daca este gol
        informatie_noua = self.informatie.copy() # trebuie copiata informatia curenta pt a nu fi suprascrisa
        informatie_noua[row, col] = self.jucator
        succesor = Nod(informatie_noua, self, Game.jucator_opus(self.jucator))
        succesori.append(succesor)
        break # trecem la coloana urmatoare
    return succesori


  def check_linii(self, jucator):
    ''' Verifica daca exista vreo linie care contine o secventa de 4'''
    for row in range(5, -1, -1):
      for col in range(4):
        secv_4 = self.informatie[row, col:col+4]
        counter = (jucator == secv_4).sum()
        if counter == 4:
          return True
    return False

  def check_coloane(self, jucator):
    ''' Verifica daca exista vreo coloane care contine o secventa de 4'''
    for col in range(7):
      for row in range(5, 2, -1):
        secv_4 = self.informatie[row-3:row+1, col]
        counter = (jucator == secv_4).sum()
        if counter == 4:
          return True
    return False

  def check_diagonale_princ(self, jucator):
    for row in range(5,2,-1):
      for col in range(3, 7):
        counter = 0
        for i in range(4):
          if self.informatie[row-i, col-i] == jucator:
            counter += 1
          else:
            counter = 0
            break
        if counter == 4:
          return True
    return False

  def check_diagonale_sec(self, jucator):
    for row in range(5,2,-1):
      for col in range(4):
        counter = 0
        for i in range(4):
          if self.informatie[row-i, col+i] == jucator:
            counter += 1
          else:
            counter = 0
            break
        if counter == 4:
          return True
    return False

  def check_jucator(self, jucator):
    ''' Verifica daca a castigat jucatorul dat'''
    return any([
        self.check_linii(jucator),
        self.check_coloane(jucator),
        self.check_diagonale_princ(jucator),
        self.check_diagonale_sec(jucator)
   ])

  def stare_scop(self):
    ''' Verifica daca nodul curent e nod scop. Daca da, returneaza castigatorul sau remiza '''
    if self.check_jucator(Game.MAX):
        return Game.MAX

    if self.check_jucator(Game.MIN):
        return Game.MIN

    if Game.GOL in self.informatie:
      return None

    return 'remiza'


  def scor_secventa_4_simboluri(self, secv, jucator):
    ''' Calculeaza scorul unei secvente de 4 simboluri la rand.
    :param secv: secventa de 4 simboluri
    :param jucator: simbolul jucatorului pentru care se calculeaza scorul
    :return: scorul jucatorului curent pe secventa data
    '''
    if Game.jucator_opus(jucator) in secv:
      return 0
    counter = (jucator == secv).sum()
    return 10 ** (counter-1) if counter > 0 else 0


  def scor_linii(self, jucator):
    scor = 0
    for row in range(5, -1, -1):
      if Game.MAX not in self.informatie[row] and Game.MIN not in self.informatie[row]: # linie goala
        break
      for col in range(4):
        secv_4 = self.informatie[row, col:col+4]
        secv_valida = True
        if row < 5:
          suport = self.informatie[row+1, col:col+4]
          if Game.GOL in suport:
            secv_valida = False
        if secv_valida:
          scor += self.scor_secventa_4_simboluri(secv_4, jucator)
    return scor


  def scor_coloane(self, jucator):
    scor = 0
    for col in range(7):
      for row in range(5, 2, -1):
        secv_4 = self.informatie[row-3:row+1, col]
        if Game.jucator_opus(jucator) not in secv_4:
          # calculam scorul pentru o singura secventa deoarece putem pune un singut disc intr-o coloana
          scor += self.scor_secventa_4_simboluri(secv_4, jucator)
          break
    return scor


  def scor_diagonale_princ(self, jucator):
    scor = 0
    for row in range(5,2,-1):
      for col in range(3, 7):
        secv = []
        for i in range(4):
          if row-i == 5 or self.informatie[row-i+1, col-i] != Game.GOL: # verifcam ca diagonala este valida (are baza)
            secv.append(self.informatie[row-i, col-i])
          else:
            break
        if len(secv) == 4:
          scor += self.scor_secventa_4_simboluri(np.array(secv), jucator)
    return scor


  def scor_diagonale_sec(self, jucator):
    scor = 0
    for row in range(5,2,-1):
      for col in range(4):
        secv = []
        for i in range(4):
          if row-i == 5 or self.informatie[row-i+1, col+i] != Game.GOL: # verifcam ca diagonala este valida (are baza)
            secv.append(self.informatie[row-i, col+i])
          else:
            break
        if len(secv) == 4:
          scor += self.scor_secventa_4_simboluri(np.array(secv), jucator)
    return scor


  def scor_jucator(self, jucator):
    return int(self.scor_linii(jucator) \
               + self.scor_coloane(jucator) \
               + self.scor_diagonale_princ(jucator) \
               + self.scor_diagonale_sec(jucator))

  def scor_stare(self):
    scor_max = self.scor_jucator(Game.MAX)
    scor_min = self.scor_jucator(Game.MIN)
    return scor_max - scor_min

In [ ]:
class Graf:
  def __init__(self):
    ''' Constructorul clasei Graf. '''
    self.nodStart = Nod()

  def succesori(self, nod):
    ''' Primeste un nod al arborelui de parcurgere si returnează lista
    succesorilor directi ai nodului care nu au fost vizitati pe ramura curenta.

    :param nod: un nod oarecare
    :return: lista succesorilor valizi ai nodului
    '''
    succesori = nod.succesori()
    for nodCurent in succesori:
      nodCurent.parinte = nod
      #nod.lista_succesori.append(nodCurent)
    return succesori


  def alpha_beta(self, nod, joacaMax, alpha, beta, adancime):
    if adancime == 0 or nod.stare_scop():
      nod.estimare_scor = nod.scor_stare()
      return nod

    succesori = self.succesori(nod)

    if joacaMax:
      nod_best = Nod()
      nod_best.estimare_scor = -np.inf
      for succesor in succesori:
        nod_evaluat = self.alpha_beta(succesor, not joacaMax, alpha, beta, adancime-1)
        if nod_evaluat.estimare_scor > nod_best.estimare_scor:
          nod_best = nod_evaluat
        if alpha < nod_evaluat.estimare_scor:
          alpha = nod_evaluat.estimare_scor
        if beta <= alpha:
          break
    else:
      nod_best = Nod()
      nod_best.estimare_scor = np.inf
      for succesor in succesori:
        nod_evaluat = self.alpha_beta(succesor, not joacaMax, alpha, beta, adancime-1)
        if nod_evaluat.estimare_scor < nod_best.estimare_scor:
          nod_best = nod_evaluat
        if beta > nod_evaluat.estimare_scor:
          beta = nod_evaluat.estimare_scor
        if beta <= alpha:
          break

    return nod_best

In [ ]:
graf = Graf()
nod = graf.alpha_beta(graf.nodStart, True, -np.inf, np.inf, 5)
print(nod) # starea terminala cea mai avantajoasa pentru MAX de la nivelul 5
print(nod.estimare_scor)

.|.|.|.|.|.|.
-------------
.|.|.|.|.|.|.
-------------
.|.|.|.|.|.|.
-------------
.|.|R|.|.|.|.
-------------
.|.|R|.|.|.|.
-------------
.|.|Y|R|Y|.|.
10
